# Databricks Volume Disaster Recovery (DR) Sync Script

## Overview
This notebook provides an automated disaster recovery solution for synchronizing Databricks Unity Catalog volumes between source and target catalogs. The script performs efficient, incremental file synchronization with comprehensive logging and parallel processing capabilities.

## Key Features
- **Incremental Sync**: Only copies new or modified files based on modification timestamps
- **Parallel Processing**: Uses ThreadPoolExecutor for concurrent volume synchronization
- **Comprehensive Logging**: Tracks all operations with detailed status and metadata in a log table
- **Recursive Directory Support**: Handles nested directory structures automatically
- **Error Handling**: Robust exception handling with detailed error logging
- **Unity Catalog Integration**: Creates target catalogs, schemas, and volumes as needed

## Configuration Parameters

| Parameter | Description | Example |
|-----------|-------------|---------|
| `catalog_names` | Comma-separated source:target catalog pairs | `source_cat:target_cat,cat2:dr_cat2` |
| `target_schema_owner` | Schema ownership assignments for target catalogs | `target_cat.schema:owner_group` |
| `max_workers` | Maximum number of concurrent threads for parallel processing | `5` |
| `log_table_name` | Full name of the logging table to track operations | `users.admin.dr_log_table` |

## How It Works

1. **Initialization**: Creates log table and parses configuration parameters
2. **Catalog Processing**: For each source:target catalog pair:
   - Creates target catalog if it doesn't exist
   - Iterates through all schemas in the source catalog
3. **Schema Processing**: For each schema:
   - Creates corresponding target schema
   - Identifies all volumes within the schema
4. **Volume Synchronization**: For each volume:
   - Creates target volume if needed
   - Retrieves previous run information for incremental sync
   - Recursively copies only new/modified files
   - Logs all operations with timestamps and metadata

## Prerequisites
- Unity Catalog enabled workspace
- Appropriate permissions on source and target catalogs
- Access to create and write to the specified log table
- Databricks Runtime 13.3 LTS or above

## Usage Notes
- **First Run**: Copies all files from source to target volumes
- **Subsequent Runs**: Only copies files that are new or have been modified
- **Logging**: All operations are logged for auditing and troubleshooting
- **Error Recovery**: Failed operations are logged but don't stop the entire sync process

## Important Considerations
- Ensure sufficient cluster resources for large volume synchronizations
- Monitor the log table for any failed operations
- Test with small volumes before running on production data
- Consider network bandwidth and storage costs for large data transfers

---
**Created**: September 2025  
**Version**: 1.0  
**Runtime Compatibility**: Databricks Runtime 13.3 LTS+



In [0]:
# dbutils.widgets.removeAll()
dbutils.widgets.text("catalog_names", "")#source and target catalog names like dictionary
dbutils.widgets.text("target_schema_owner", "")
dbutils.widgets.text("max_workers", "5")
dbutils.widgets.text("log_table_name", "users.satyendranath_sure.dr_log_table_name")

In [0]:
catalog_pair_names_list = [catalog_pair.strip() for catalog_pair in dbutils.widgets.get("catalog_names").split(",")]
target_schema_owner_list = [catalog_schema_pair.strip() for catalog_schema_pair in dbutils.widgets.get("target_schema_owner").split(",")]
max_workers = int(dbutils.widgets.get("max_workers"))
log_table_name = dbutils.widgets.get("log_table_name")
print(catalog_pair_names_list, target_schema_owner_list, max_workers, log_table_name)

In [0]:
# for catalog_pair in catalog_pair_names_list:
#     second_value = catalog_pair.split(":")[1]
#     print(second_value)
#     spark.sql(f"DROP CATALOG IF EXISTS `{second_value}` CASCADE")
# spark.sql(f"DROP table if exists `{log_table_name}`")

In [0]:

import traceback
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, ArrayType
from datetime import datetime
import json
from pyspark.sql import SparkSession
import os

# The 'spark' and 'dbutils' objects are pre-defined in a Databricks environment.

# Get the current notebook name
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_name = notebook_path.split('/')[-1]

# Schema definition for log table
log_table_schema = StructType([
    StructField("notebook_name", StringType(), True),
    StructField("entity_type", StringType(), True),
    StructField("entity_name", StringType(), True),
    StructField("action", StringType(), True),
    StructField("status", StringType(), True),
    StructField("message", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("results_data", ArrayType(StructType([
        StructField("key", StringType(), True),
        StructField("value", StringType(), True)
    ])), True)
])

# Create log table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {log_table_name}
    (
        notebook_name STRING,
        entity_type STRING,
        entity_name STRING,
        action STRING,
        status STRING,
        message STRING,
        timestamp TIMESTAMP,
        results_data ARRAY<STRUCT<key: STRING, value: STRING>>
    )
""")

def insert_log_entry(log_data):
    """
    Inserts a log entry into the designated log table.
    
    Args:
        log_data (dict): A dictionary containing log information
    """
    try:
        log_data["notebook_name"] = notebook_name
        # Ensure results_data is a list of dictionaries as per schema
        if "results_data" not in log_data or not isinstance(log_data["results_data"], list):
            log_data["results_data"] = []
        
        df = spark.createDataFrame([log_data], schema=log_table_schema)
        df.write.mode("append").saveAsTable(log_table_name)
    except Exception as e:
        print(f"Failed to insert log entry: {log_data}. Error: {e}")

def get_previous_run_info(source_path):
    """
    Retrieves the modification times of files from the last successful run for a given source path.
    
    This is used to identify which files are new or have been modified since the last copy.
    
    Args:
        source_path (str): The source path (directory) to check for previously copied files.
    
    Returns:
        dict: A dictionary mapping absolute file paths to their last successful modification timestamp.
    """
    spark_session = SparkSession.builder.getOrCreate()
    try:
        df = spark_session.sql(f"""
            SELECT
                entity_name,
                results_data
            FROM {log_table_name}
            WHERE
                entity_type = 'file' AND
                action = 'copy' AND
                status = 'success'
            ORDER BY timestamp DESC
        """)
        
        previous_runs = {}
        for row in df.collect():
            try:
                file_path = row['entity_name']
                results_data = row['results_data']
                mod_time = None
                for item in results_data:
                    if item.key == 'modificationTime':
                        mod_time = int(item.value)
                        break
                
                if mod_time:
                    # Use absolute file path instead of relative path
                    if file_path not in previous_runs:
                        previous_runs[file_path] = mod_time
            except Exception as e:
                print(f"Error parsing log entry: {e}")
                continue
        return previous_runs
    except Exception as e:
        log_entry = {
            "entity_type": "info",
            "entity_name": source_path,
            "action": "get_previous_run_info",
            "status": "error",
            "message": f"Error fetching previous run info. Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        }
        insert_log_entry(log_entry)
        return {}


def sync_volume_recursively(source_path, target_path, prev_run_info):
    """
    Recursively copies files from a source path to a target path,
    copying only new or modified files.
    
    Args:
        source_path (str): The full source path (dbfs:/Volumes/...) to sync from.
        target_path (str): The full target path (dbfs:/Volumes/...) to sync to.
        prev_run_info (dict): A dictionary of previously copied files and their modification times.
    """
    # Ensure the target directory exists before copying files into it.
    try:
        dbutils.fs.mkdirs(target_path)
    except Exception as e:
        log_entry = {
            "entity_type": "directory",
            "entity_name": target_path,
            "action": "create",
            "status": "error",
            "message": f"Failed to create target directory. Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        }
        insert_log_entry(log_entry)

    try:
        # List contents of the source path.
        files = dbutils.fs.ls(source_path)
    except Exception as e:
        log_entry = {
            "entity_type": "directory",
            "entity_name": source_path,
            "action": "list",
            "status": "error",
            "message": f"Error listing contents of directory. Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        }
        insert_log_entry(log_entry)
        return

    for file_info in files:
        # Check if the item is a directory by looking for a trailing slash in its name
        if file_info.name.endswith('/'):
            # It's a directory, so we need to process it recursively.
            log_entry = {
                "entity_type": "directory",
                "entity_name": file_info.path,
                "action": "list",
                "status": "success",
                "message": "Found directory to traverse.",
                "timestamp": datetime.now(),
                "results_data": []
            }
            insert_log_entry(log_entry)
            # Construct the new target path correctly for the subdirectory
            new_target_path = os.path.join(target_path, file_info.name)
            sync_volume_recursively(file_info.path, new_target_path, prev_run_info)
        else:
            # It's a file. Check if it needs to be copied.
            relative_path = file_info.path.replace(source_path, '', 1).lstrip('/')
            
            # Compare modification times.
            if file_info.path not in prev_run_info or file_info.modificationTime > prev_run_info[file_info.path]:
                try:
                    # Attempt to copy the file.
                    # The dbutils.fs.cp command copies the source file into the destination directory.
                    dbutils.fs.cp(file_info.path, target_path, recurse=True)
                    
                    # Log successful copy with file modification time.
                    results_data = [
                        {"key": "result", "value": "True"},
                        {"key": "modificationTime", "value": str(file_info.modificationTime)},
                        {"key": "fileSize", "value": str(file_info.size)},
                        {"key": "source_path", "value": file_info.path},
                        {"key": "target_path", "value": target_path}
                    ]
                    log_entry = {
                        "entity_type": "file",
                        "entity_name": file_info.path,
                        "action": "copy",
                        "status": "success",
                        "message": "File copied successfully.",
                        "timestamp": datetime.now(),
                        "results_data": results_data
                    }
                    insert_log_entry(log_entry)
                except Exception as e:
                    # Log a failure to copy the file.
                    log_entry = {
                        "entity_type": "file",
                        "entity_name": file_info.path,
                        "action": "copy",
                        "status": "error",
                        "message": f"Error during file copy. Error: {e}",
                        "timestamp": datetime.now(),
                        "results_data": []
                    }
                    insert_log_entry(log_entry)
            else:
                log_entry = {
                    "entity_type": "file",
                    "entity_name": file_info.path,
                    "action": "skip",
                    "status": "success",
                    "message": "File already exists with same or newer modification date. Skipping.",
                    "timestamp": datetime.now(),
                    "results_data": []
                }
                insert_log_entry(log_entry)

def execute_and_log_sql(sql_command, entity_type, entity_name, action, success_msg, error_msg):
    """
    Executes a SQL command and logs the result.
    
    Args:
        sql_command (str): The SQL command to execute.
        entity_type (str): The type of entity being acted upon (e.g., 'catalog', 'schema').
        entity_name (str): The name of the entity.
        action (str): The action being performed (e.g., 'create').
        success_msg (str): Message to log on success.
        error_msg (str): Message to log on error.
    """
    spark_session = SparkSession.builder.getOrCreate()
    try:
        spark_session.sql(sql_command)
        log_entry = {
            "entity_type": entity_type,
            "entity_name": entity_name,
            "action": action,
            "status": "success",
            "message": success_msg,
            "timestamp": datetime.now(),
            "results_data": []
        }
        insert_log_entry(log_entry)
    except Exception as e:
        log_entry = {
            "entity_type": entity_type,
            "entity_name": entity_name,
            "action": action,
            "status": "error",
            "message": f"{error_msg} Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        }
        insert_log_entry(log_entry)


def process_volume_sync(source_catalog, source_schema, target_catalog, target_schema, volume_name):
    """
    Processes the synchronization for a single volume.
    
    Args:
        source_catalog (str): The source catalog name.
        source_schema (str): The source schema name.
        target_catalog (str): The target catalog name.
        target_schema (str): The target schema name.
        volume_name (str): The name of the volume to sync.
    """
    print(f"Starting sync for volume: {source_catalog}.{source_schema}.{volume_name}")
    spark_session = SparkSession.builder.getOrCreate()
    try:
        # 1. Create the target volume if it doesn't exist.
        sql_command = f"CREATE VOLUME IF NOT EXISTS `{target_catalog}`.`{target_schema}`.`{volume_name}`"
        try:
            spark_session.sql(sql_command)
            log_entry = {
                "entity_type": "volume",
                "entity_name": f"{target_catalog}.{target_schema}.{volume_name}",
                "action": "create",
                "status": "success",
                "message": "Volume created or already exists.",
                "timestamp": datetime.now(),
                "results_data": []
            }
            insert_log_entry(log_entry)
        except Exception as e:
            log_entry = {
                "entity_type": "volume",
                "entity_name": f"{target_catalog}.{target_schema}.{volume_name}",
                "action": "create",
                "status": "error",
                "message": f"Failed to create volume. Error: {e}",
                "timestamp": datetime.now(),
                "results_data": []
            }
            insert_log_entry(log_entry)
            return

        # 2. Construct source and target paths for file system operations.
        source_path = os.path.join("/Volumes", source_catalog, source_schema, volume_name)
        target_path = os.path.join("/Volumes", target_catalog, target_schema, volume_name)
        
        # 3. Get modification dates from previous runs to enable efficient sync.
        prev_run_info = get_previous_run_info(source_path)

        # 4. Start the recursive file sync.
        sync_volume_recursively(source_path, target_path, prev_run_info)
        
    except Exception as e:
        log_entry = {
            "entity_type": "volume_sync",
            "entity_name": f"{source_catalog}.{source_schema}.{volume_name}",
            "action": "sync",
            "status": "error",
            "message": f"An unexpected error occurred during volume sync. Error: {e}",
            "timestamp": datetime.now(),
            "results_data": []
        }
        insert_log_entry(log_entry)

# ==============================================================================
# MAIN EXECUTION LOGIC
# ==============================================================================
if __name__ == "__main__":
    spark = SparkSession.builder.getOrCreate()
    spark.sql("SET spark.sql.catalog.spark_catalog = 'spark_catalog'")
    
    print("Starting Databricks Volume DR Sync.")
    
    log_entry = {
        "entity_type": "info",
        "entity_name": "N/A",
        "action": "start_sync",
        "status": "success",
        "message": "Starting Databricks Volume DR Sync.",
        "timestamp": datetime.now(),
        "results_data": []
    }
    insert_log_entry(log_entry)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []

        for catalog_pair in catalog_pair_names_list:
            try:
                source_catalog, target_catalog = catalog_pair.split(':')
            except ValueError:
                log_entry = {
                    "entity_type": "info",
                    "entity_name": catalog_pair,
                    "action": "parse_catalog_pair",
                    "status": "error",
                    "message": f"Invalid catalog pair format: {catalog_pair}. Skipping.",
                    "timestamp": datetime.now(),
                    "results_data": []
                }
                insert_log_entry(log_entry)
                continue

            try:
                # 1. Create target catalog if it doesn't exist.
                execute_and_log_sql(
                    f"CREATE CATALOG IF NOT EXISTS `{target_catalog}`",
                    "catalog",
                    target_catalog,
                    "create",
                    f"Catalog `{target_catalog}` created or already exists.",
                    f"Failed to create catalog `{target_catalog}`."
                )
                
                # 2. Get list of schemas in the source catalog.
                schemas_df = spark.sql(f"SHOW SCHEMAS IN `{source_catalog}`")
                schemas_to_process = [row['databaseName'] for row in schemas_df.collect()]
                
                for source_schema in schemas_to_process:
                    # 3. Ignore 'information_schema'.
                    if source_schema.lower() == 'information_schema':
                        log_entry = {
                            "entity_type": "schema",
                            "entity_name": f"{source_catalog}.{source_schema}",
                            "action": "skip",
                            "status": "success",
                            "message": "Skipping 'information_schema'.",
                            "timestamp": datetime.now(),
                            "results_data": []
                        }
                        insert_log_entry(log_entry)
                        continue

                    try:
                        # 4. Create target schema if it doesn't exist.
                        execute_and_log_sql(
                            f"CREATE SCHEMA IF NOT EXISTS `{target_catalog}`.`{source_schema}`",
                            "schema",
                            f"{target_catalog}.{source_schema}",
                            "create",
                            f"Schema `{source_schema}` in catalog `{target_catalog}` created or already exists.",
                            f"Failed to create schema `{source_schema}`."
                        )
                        
                        # 5. Get list of volumes in the source schema.
                        volumes_df = spark.sql(f"SHOW VOLUMES IN `{source_catalog}`.`{source_schema}`")
                        volumes_to_process = [row['volume_name'] for row in volumes_df.collect()]
                        
                        for volume_name in volumes_to_process:
                            # 6. Submit volume sync tasks to the thread pool.
                            future = executor.submit(
                                process_volume_sync,
                                source_catalog,
                                source_schema,
                                target_catalog,
                                source_schema, # Use source_schema as target_schema for simplicity
                                volume_name
                            )
                            futures.append(future)

                    except Exception as e:
                        log_entry = {
                            "entity_type": "schema",
                            "entity_name": f"{source_catalog}.{source_schema}",
                            "action": "process",
                            "status": "error",
                            "message": f"An error occurred while processing schema. Error: {e}",
                            "timestamp": datetime.now(),
                            "results_data": []
                        }
                        insert_log_entry(log_entry)

            except Exception as e:
                log_entry = {
                    "entity_type": "catalog",
                    "entity_name": source_catalog,
                    "action": "process",
                    "status": "error",
                    "message": f"An error occurred while processing catalog pair. Error: {e}",
                    "timestamp": datetime.now(),
                    "results_data": []
                }
                insert_log_entry(log_entry)

        # Wait for all tasks to complete and retrieve results.
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                log_entry = {
                    "entity_type": "info",
                    "entity_name": "N/A",
                    "action": "future_result",
                    "status": "error",
                    "message": f"An unexpected error occurred in a thread. Error: {e}",
                    "timestamp": datetime.now(),
                    "results_data": []
                }
                insert_log_entry(log_entry)
                
    print("Databricks Volume DR Sync completed.")

    log_entry = {
        "entity_type": "info",
        "entity_name": "N/A",
        "action": "end_sync",
        "status": "success",
        "message": "Databricks Volume DR Sync completed.",
        "timestamp": datetime.now(),
        "results_data": []
    }
    insert_log_entry(log_entry)





In [0]:
# spark.sql(f"""
#     DELETE
#     FROM {log_table_name}
#     WHERE notebook_name = '{notebook_name}'
# """)

In [0]:
# Display logs for the last 24 hours
print(f"\n--- Displaying logs from {log_table_name} for the last 24 hours ---")
# past_24_hours = datetime.now() - timedelta(hours=24)
log_query_df = spark.sql(f"""
    SELECT *
    FROM {log_table_name} where notebook_name = '{notebook_name}'
    ORDER BY timestamp DESC
""")

display(log_query_df)

In [0]:
# %sql
# drop table 
# -- select * from 
# -- users.satyendranath_sure.dr_log_table_name